In [17]:
# Model

from ortools.linear_solver import pywraplp

# Objective Function: Auxiliary Functions
teu = {
    'L': 6.096,
    'H': 2.591,
    'W': 2.438
}

# speeds in m/min
speeds = {
    'gantry': {
        'empty': 90,
        'loaded': 45
    },
    'trolley': {
        'empty': 70,
        'loaded': 70
    },
    'hoisting': {
        'empty': 24,
        'loaded': 12
    }
}

def expected_ct_receiving(L, H, W, dim=teu, speeds=speeds):
    gantry_range = {
        'min': 0,
        'max': L * dim.get('L')
    }
    trolley_range = {
        'min': 0,
        'max': W * dim.get('W')
    }
    hoisting_range = {
        'min': 0,
        'max': 15.4
    }

    times = []

    # 1: Max(T-rcte, T-rrge)
    e_D_rcte = (trolley_range.get('min') + trolley_range.get('max')) / 2
    e_T_rcte = e_D_rcte / speeds.get('trolley').get('empty')

    e_D_rrge = (gantry_range.get('min') + gantry_range.get('max')) / 2
    e_T_rrge = e_D_rrge / speeds.get('gantry').get('empty')

    times.append(e_T_rcte if e_T_rcte >= e_T_rrge else e_T_rrge)

    # 2: t-tche
    e_t_tche = (hoisting_range.get('max')) / speeds.get('hoisting').get('empty')

    times.append(e_t_tche)

    # 3: t-p
    e_t_p = 5 / 60 # 5 seconds

    times.append(e_t_p)

    # 4: t-cthl
    e_t_cthl = (hoisting_range.get('max')) / speeds.get('hoisting').get('loaded')

    times.append(e_t_cthl)

    # 5: T-crtl
    e_D_crtl = (trolley_range.get('min') + trolley_range.get('max')) / 2
    e_T_crtl = e_D_crtl / speeds.get('trolley').get('loaded')

    times.append(e_T_crtl)

    # 6: T-trhl
    e_D_trhl = (hoisting_range.get('min') + hoisting_range.get('max')) / 2
    e_T_trhl = e_D_trhl / speeds.get('hoisting').get('loaded')

    times.append(e_T_trhl)

    # 7: t-r
    e_t_r = 5 / 60 # 5 seconds

    times.append(e_t_r)

    # 8: T-rthe
    e_D_rthe = (hoisting_range.get('min') + hoisting_range.get('max')) / 2
    e_T_rthe = e_D_rthe / speeds.get('hoisting').get('empty')

    times.append(e_T_rthe)
    
    return sum(times)

def expected_ct_loading(L, H, W, dim=teu, speeds=speeds):
    gantry_range = {
        'min': 0,
        'max': L * dim.get('L')
    }
    trolley_range = {
        'min': 0,
        'max': W * dim.get('W')
    }
    hoisting_range = {
        'min': 0,
        'max': 15.4
    }

    times = []

    # 1: Max(T-crte, T-rrge) (f=1/l0)
    l0 = 10 # Assumption
    e_D_crte = (trolley_range.get('min') + trolley_range.get('max')) / 2
    e_T_crte = e_D_crte / speeds.get('trolley').get('empty')

    e_D_rrge = (gantry_range.get('min') + gantry_range.get('max')) / 2
    e_T_rrge = e_D_rrge / speeds.get('gantry').get('empty')

    times.append((e_T_crte if e_T_crte >= e_T_rrge else e_T_rrge) * (1 / l0))

    # 2: T-trhe
    e_D_trhe = (hoisting_range.get('min') + hoisting_range.get('max')) / 2
    e_T_trhe = e_D_trhe / speeds.get('hoisting').get('empty')

    times.append(e_T_trhe)

    # 3: t-p
    e_t_p = 5 / 60 # 5 seconds

    times.append(e_t_p)

    # 4: T-rthl
    e_D_rthl = (hoisting_range.get('min') + hoisting_range.get('max')) / 2
    e_T_rthl = e_D_rthl / speeds.get('hoisting').get('loaded')

    times.append(e_T_rthl)

    # 5: T-rctl
    e_D_rctl = (trolley_range.get('min') + trolley_range.get('max')) / 2
    e_T_rctl = e_D_rctl / speeds.get('trolley').get('loaded')

    times.append(e_T_rctl)

    # 6: t-tchl
    e_D_tchl = hoisting_range.get('max')
    e_t_tchl = e_D_tchl / speeds.get('hoisting').get('loaded')

    times.append(e_t_tchl)

    # 7: t-r
    e_t_r = 5 / 60 # 5 seconds

    times.append(e_t_r)

    # 8: t-cthe
    e_D_cthe = hoisting_range.get('max')
    e_t_cthe = e_D_cthe / speeds.get('hoisting').get('empty')

    times.append(e_t_cthe)

    # 9: T-crte (f=(l0-1)/l0)
    l0 = 10 # Assumption
    e_D_crte = (trolley_range.get('min') + trolley_range.get('max')) / 2
    e_T_crte = e_D_crte / speeds.get('trolley').get('empty')

    times.append(e_T_crte * (l0-1) / l0)

    return sum(times)

# Parameters
bounds_L = {'min': 1, 'max': 100}
bounds_H = {'min': 1, 'max': 8}
bounds_W = {'min': 1, 'max': 100}
min_storage = 6 * 5 * 16
alphas = [0.5, 0.6, 0.7, 0.8, 0.9]

results = []
for alpha in alphas:
    alpha_result = []
    for L in range(bounds_L.get('min'), bounds_L.get('max') + 1):
        for H in range(bounds_H.get('min'), bounds_H.get('max') + 1):
            for W in range(bounds_W.get('min'), bounds_W.get('max') + 1):
                alpha_result.append({'configuration': (L, H, W), 'time_value': alpha * expected_ct_loading(L, H, W) + (1 - alpha) * expected_ct_receiving(L, H, W)})
    results.append({'alpha': alpha, 'results': alpha_result})

import json

with open('model-results.json', 'w') as f:
    json.dump(results, f, indent=4)


In [10]:
# Gantry measurements
# http://www.gantrycranedesign.com/rubber-tyred-gantry-crane.html

# YC Movements AuxFuncs
# s = d/t, d = ts, t = d/s
# dists in m
teu = {
    'L': 6.096,
    'H': 2.591,
    'W': 2.438
}

# speeds in m/min
speeds = {
    'gantry': {
        'empty': 90,
        'loaded': 45
    },
    'trolley': {
        'empty': 70,
        'loaded': 70
    },
    'hoisting': {
        'empty': 24,
        'loaded': 12
    }
}

def expected_ct_receiving(L, H, W, dim=teu, speeds=speeds):
    gantry_range = {
        'min': 0,
        'max': L * dim.get('L')
    }
    trolley_range = {
        'min': 0,
        'max': W * dim.get('W')
    }
    hoisting_range = {
        'min': 0,
        'max': 15.4
    }

    times = []

    # 1: Max(T-rcte, T-rrge)
    e_D_rcte = (trolley_range.get('min') + trolley_range.get('max')) / 2
    e_T_rcte = e_D_rcte / speeds.get('trolley').get('empty')

    e_D_rrge = (gantry_range.get('min') + gantry_range.get('max')) / 2
    e_T_rrge = e_D_rrge / speeds.get('gantry').get('empty')

    times.append(max(e_T_rcte, e_T_rrge))

    # 2: t-tche
    e_t_tche = (hoisting_range.get('max')) / speeds.get('hoisting').get('empty')

    times.append(e_t_tche)

    # 3: t-p
    e_t_p = 5 / 60 # 5 seconds

    times.append(e_t_p)

    # 4: t-cthl
    e_t_cthl = (hoisting_range.get('max')) / speeds.get('hoisting').get('loaded')

    times.append(e_t_cthl)

    # 5: T-crtl
    e_D_crtl = (trolley_range.get('min') + trolley_range.get('max')) / 2
    e_T_crtl = e_D_crtl / speeds.get('trolley').get('loaded')

    times.append(e_T_crtl)

    # 6: T-trhl
    e_D_trhl = (hoisting_range.get('min') + hoisting_range.get('max')) / 2
    e_T_trhl = e_D_trhl / speeds.get('hoisting').get('loaded')

    times.append(e_T_trhl)

    # 7: t-r
    e_t_r = 5 / 60 # 5 seconds

    times.append(e_t_r)

    # 8: T-rthe
    e_D_rthe = (hoisting_range.get('min') + hoisting_range.get('max')) / 2
    e_T_rthe = e_D_rthe / speeds.get('hoisting').get('empty')

    times.append(e_T_rthe)
    
    return times

def expected_ct_loading(L, H, W, dim=teu, speeds=speeds):
    gantry_range = {
        'min': 0,
        'max': L * dim.get('L')
    }
    trolley_range = {
        'min': 0,
        'max': W * dim.get('W')
    }
    hoisting_range = {
        'min': 0,
        'max': 15.4
    }

    times = []

    # 1: Max(T-crte, T-rrge) (f=1/l0)
    l0 = 10 # Assumption
    e_D_crte = (trolley_range.get('min') + trolley_range.get('max')) / 2
    e_T_crte = e_D_crte / speeds.get('trolley').get('empty')

    e_D_rrge = (gantry_range.get('min') + gantry_range.get('max')) / 2
    e_T_rrge = e_D_rrge / speeds.get('gantry').get('empty')

    times.append(max(e_T_crte, e_T_rrge) * (1 / l0))

    # 2: T-trhe
    e_D_trhe = (hoisting_range.get('min') + hoisting_range.get('max')) / 2
    e_T_trhe = e_D_trhe / speeds.get('hoisting').get('empty')

    times.append(e_T_trhe)

    # 3: t-p
    e_t_p = 5 / 60 # 5 seconds

    times.append(e_t_p)

    # 4: T-rthl
    e_D_rthl = (hoisting_range.get('min') + hoisting_range.get('max')) / 2
    e_T_rthl = e_D_rthl / speeds.get('hoisting').get('loaded')

    times.append(e_T_rthl)

    # 5: T-rctl
    e_D_rctl = (trolley_range.get('min') + trolley_range.get('max')) / 2
    e_T_rctl = e_D_rctl / speeds.get('trolley').get('loaded')

    times.append(e_T_rctl)

    # 6: t-tchl
    e_D_tchl = hoisting_range.get('max')
    e_t_tchl = e_D_tchl / speeds.get('hoisting').get('loaded')

    times.append(e_t_tchl)

    # 7: t-r
    e_t_r = 5 / 60 # 5 seconds

    times.append(e_t_r)

    # 8: t-cthe
    e_D_cthe = hoisting_range.get('max')
    e_t_cthe = e_D_cthe / speeds.get('hoisting').get('empty')

    times.append(e_t_cthe)

    # 9: T-crte (f=(l0-1)/l0)
    l0 = 10 # Assumption
    e_D_crte = (trolley_range.get('min') + trolley_range.get('max')) / 2
    e_T_crte = e_D_crte / speeds.get('trolley').get('empty')

    times.append(e_T_crte * (l0-1) / l0)

    return times

leekim_speeds = {
    'gantry': {
        'empty': 180,
        'loaded': 180
    },
    'trolley': {
        'empty': 150,
        'loaded': 150
    },
    'hoisting': {
        'empty': 83,
        'loaded': 50
    }
}

receiving = expected_ct_receiving(46, 2, 20, speeds=leekim_speeds)
print('RECEIVING:', receiving, sum(receiving))
loading = expected_ct_loading(46, 2, 20, speeds=leekim_speeds)
print('LOADING:', loading, sum(loading))

RECEIVING: [0.7789333333333334, 0.1855421686746988, 0.08333333333333333, 0.308, 0.16253333333333336, 0.154, 0.08333333333333333, 0.0927710843373494] 1.8484465863453816
LOADING: [0.07789333333333334, 0.0927710843373494, 0.08333333333333333, 0.154, 0.16253333333333336, 0.308, 0.08333333333333333, 0.1855421686746988, 0.14628000000000002] 1.2936865863453817


In [ ]:
# INFEASIBLE: One constraint (LWH >= min_storage) is non-linear.
# for alpha in alphas:
#     # Create the mip solver with the SCIP backend.
#     solver = pywraplp.Solver.CreateSolver('SCIP')

#     # Solver 
#     L = solver.IntVar(bounds_L.get('min'), bounds_L.get('max'), 'L')
#     H = solver.IntVar(bounds_H.get('min'), bounds_H.get('max'), 'H')
#     W = solver.IntVar(bounds_W.get('min'), bounds_W.get('max'), 'W')

#     solver.Minimize(alpha * expected_ct_loading(L, H, W) + (1 - alpha) * expected_ct_receiving(L, H, W))

#     solver.Add(bounds_L.get('min') <= L <= bounds_L.get('max'))
#     solver.Add(bounds_H.get('min') <= H <= bounds_H.get('max'))
#     solver.Add(bounds_W.get('min') <= W <= bounds_W.get('max'))
#     solver.Add(L * W >= 6 * 5 * 16)

#     status = solver.Solve()

#     if status == pywraplp.Solver.OPTIMAL:
#         print('Solution:')
#         print('Objective value =', solver.Objective().Value())
#         print('L =', L.solution_value())
#         print('H =', H.solution_value())
#         print('W =', W.solution_value())
#     else:
#         print('The problem does not have an optimal solution.')

In [6]:
'''
Mixed Integer Programming Solver
ICTSI
'''

from ortools.linear_solver import pywraplp
import numpy as np
import pandas as pd

# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

# Parameters
# Weight assigned to vessel operations. 
alpha = 0.65
inv_alpha = 1 - alpha
# Block storage capacity requirement (in TEU)
sq = 175
# Set of values for L that are under consideration
set_L = {i for i in range(1, 30 + 1)}
# Set of values for H that are under consideration
set_H = {i for i in range(1, 30 + 1)}
# Set of values for W that are under consideration
set_W = {i for i in range(1, 30 + 1)}
# the cycle time that is required for the receiving operation by a YC, given L, H, and W (min)
ct_receive = 10
# the cycle time that is required for the loading operation by a YC, given L, H, and W (min)
ct_load = 10
# the cycle time that is required for the discharging operation by a YC, given L, H, and W (min)
ct_discharge = 10
# the cycle time that is required for the delivery operation by a YC, given L, H, and W (min)
ct_deliver = 10

# truck waiting time for the receiving operation of a YC, given L, H, and W (min)
truckwait_receive = 10
# truck waiting time for the loading operation of a YC, given L, H, and W (min)
truckwait_load = 10
# truck waiting time for the discharging operation of a YC, given L, H, and W (min)
truckwait_discharge = 10
# truck waiting time for the delivery operation of a YC, given L, H, and W (min)
truckwait_deliver = 10

# Decision Variables
L = solver.IntVar(0.0, max(set_L), 'L')
H = solver.IntVar(0.0, max(set_H), 'H')
W = solver.IntVar(0.0, max(set_W), 'W')

In [4]:
'''
Mixed Integer Programming Solver
Example
Source: https://developers.google.com/optimization/mip/integer_opt
'''

from ortools.linear_solver import pywraplp

# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

infinity = solver.infinity()
# x and y are integer non-negative variables.
x = solver.IntVar(0.0, infinity, 'x')
y = solver.IntVar(0.0, infinity, 'y')

print('Number of variables =', solver.NumVariables())

# x + 7 * y <= 17.5.
solver.Add(x + 7 * y <= 17.5)

# x <= 3.5.
solver.Add(x <= 3.5)

print('Number of constraints =', solver.NumConstraints())

# Maximize x + 10 * y.
solver.Maximize(x + 10 * y)

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('x =', x.solution_value())
    print('y =', y.solution_value())
else:
    print('The problem does not have an optimal solution.')

Number of variables = 2
Number of constraints = 2
Solution:
Objective value = 23.0
x = 3.0
y = 2.0
